In [2]:
import pandas as pd
path = '/rds/general/user/ms2524/home/amexproject/parquet_files/train/train_'
base_cba = pd.read_parquet(path + "base.parquet", columns=['case_id','WEEK_NUM','target'])

In [12]:
applprev_1_0 = pd.read_parquet(path +'applprev_1_0.parquet')
applprev_1_1 = pd.read_parquet(path +'applprev_1_1.parquet')
applprev_2 = pd.read_parquet(path + 'applprev_2.parquet')

In [15]:
debitcard_1 = pd.read_parquet(path +'debitcard_1.parquet')
deposit_1 = pd.read_parquet(path +'deposit_1.parquet')
other_1 = pd.read_parquet(path +'other_1.parquet')

In [17]:
applprev_1_0.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D',
       'byoccupationinc_3656910L', 'cancelreason_3545846M', 'childnum_21L',
       'creationdate_885D', 'credacc_actualbalance_314A',
       'credacc_credlmt_575A', 'credacc_maxhisbal_375A',
       'credacc_minhisbal_90A', 'credacc_status_367L',
       'credacc_transactions_402L', 'credamount_590A', 'credtype_587L',
       'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A',
       'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'education_1138M',
       'employedfrom_700D', 'familystate_726L', 'firstnonzeroinstldate_307D',
       'inittransactioncode_279L', 'isbidproduct_390L', 'isdebitcard_527L',
       'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'num_group1',
       'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M',
       'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M',
       'revolvingaccount_394A', 'status_219L', 'tenor_203L'],
      dtype='object')

In [20]:
import pandas as pd

def missing_value_percentages(df):
    missing_percent = df.isnull().mean() * 100
    return missing_percent.sort_values(ascending=False)
def detect_high_cardinality(df, threshold=50):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    high_card = [col for col in cat_cols if df[col].nunique() > threshold]
    return high_card


In [21]:

detect_high_cardinality(applprev_1_0)

['approvaldate_319D',
 'cancelreason_3545846M',
 'creationdate_885D',
 'dateactivated_425D',
 'district_544M',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'employedfrom_700D',
 'firstnonzeroinstldate_307D',
 'profession_152M']

In [19]:
missing_value_percentages(applprev_1_0)

revolvingaccount_394A          95.970583
credacc_minhisbal_90A          95.674083
credacc_status_367L            95.674083
credacc_maxhisbal_375A         95.674083
credacc_transactions_402L      95.674083
credacc_actualbalance_314A     95.674083
isdebitcard_527L               93.565989
byoccupationinc_3656910L       74.492268
dtlastpmt_581D                 73.575296
dtlastpmtallstes_3545839D      62.611956
employedfrom_700D              56.096869
childnum_21L                   50.258534
dateactivated_425D             47.449896
maxdpdtolerance_577P           46.747061
approvaldate_319D              45.426043
outstandingdebt_522A           32.871036
currdebt_94A                   32.676961
familystate_726L               32.029378
firstnonzeroinstldate_307D      9.393125
tenor_203L                      8.046770
pmtnum_8L                       8.046770
annuity_853A                    4.008839
credamount_590A                 3.172300
credtype_587L                   3.172300
downpmt_134A    

In [42]:
applprev_1_0[applprev_1_0["case_id"]==20]

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
19,20,0.0,3979.4001,2018-11-02,NaN,a55475b1,NaN,2018-11-02,NaN,0.0,...,0,35748.6,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,A,12.0
20,20,0.0,4704.8003,None,NaN,P94_109_143,0.0,2018-01-16,NaN,0.0,...,1,NaN,30.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,30.0


In [24]:
applprev_1_0.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D',
       'byoccupationinc_3656910L', 'cancelreason_3545846M', 'childnum_21L',
       'creationdate_885D', 'credacc_actualbalance_314A',
       'credacc_credlmt_575A', 'credacc_maxhisbal_375A',
       'credacc_minhisbal_90A', 'credacc_status_367L',
       'credacc_transactions_402L', 'credamount_590A', 'credtype_587L',
       'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A',
       'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'education_1138M',
       'employedfrom_700D', 'familystate_726L', 'firstnonzeroinstldate_307D',
       'inittransactioncode_279L', 'isbidproduct_390L', 'isdebitcard_527L',
       'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'num_group1',
       'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M',
       'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M',
       'revolvingaccount_394A', 'status_219L', 'tenor_203L'],
      dtype='object')